In [ ]:
import os
import pandas as pd
import networkx as nx
import json

TIME_TABLE_PATH = "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/path_algorithm/dataset/timetables.csv"
TRAIN_STATIONS_PATH = "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/path_algorithm/dataset/liste-des-gares.csv"
GARES_VOYAGEURS_PATH = "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/path_algorithm/dataset/gares-de-voyageurs.csv"
TIME_TABLE_FORMATTED_PATH = "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/path_algorithm/dataset/timetables_traite.csv"

PREDEFINED_START = "Le Havre"
PREDEFINED_END = "Paris-St-Lazare"

for file_path in [TIME_TABLE_PATH, TRAIN_STATIONS_PATH, GARES_VOYAGEURS_PATH]:
    if not os.path.exists(file_path):
        print(f"Erreur : Le fichier {file_path} n'existe pas.")
        exit()

class FormatTimetables:
    def __init__(self):
        self.df = pd.read_csv(TIME_TABLE_PATH, sep="\t")
        self.train_stations = pd.read_csv(TRAIN_STATIONS_PATH, sep=";")
        self.gares_voyageurs = pd.read_csv(GARES_VOYAGEURS_PATH, sep=";")
        self.special_cases = {} 
    
    def set_columns(self):
        """Ajoute les colonnes nécessaires pour les villes de départ et d'arrivée."""
        self.df["departure_city"] = None
        self.df["arrival_city"] = None
    
    def match_station_to_city(self, station_name):
        """Tente de trouver la ville associée à une gare."""
        match = self.train_stations[self.train_stations["LIBELLE"].str.contains(station_name, case=False, regex=False)]
        if not match.empty:
            return match.iloc[0]["COMMUNE"]

        alt_match = self.gares_voyageurs[self.gares_voyageurs["Nom"].str.contains(station_name, case=False, regex=False)]
        if not alt_match.empty:
            return alt_match.iloc[0]["Code commune"]

        return None

    def match_train_stations(self):
        """Associe les gares aux villes correspondantes dans le fichier des horaires."""
        for index, row in self.df.iterrows():
            for col in ["departure", "arrival"]:
                station_name = row["trajet"].split(" - ")[0] if col == "departure" else row["trajet"].split(" - ")[1]
                station_name = self.special_cases.get(station_name, station_name)
                city = self.match_station_to_city(station_name)
                if city:
                    self.df.at[index, f"{col}_city"] = city
                else:
                    print(f"Aucune correspondance trouvée pour la gare : {station_name}")

    def format_timetables(self):
        """Formate le fichier des horaires."""
        self.set_columns()
        self.match_train_stations()
        self.df.to_csv(TIME_TABLE_FORMATTED_PATH, sep="\t", index=False)
        print(f"Fichier formaté créé : {TIME_TABLE_FORMATTED_PATH}")

class PathFinder:
    def __init__(self):
        self.df = pd.read_csv(TIME_TABLE_FORMATTED_PATH, sep="\t")
        self.graph = nx.DiGraph()
    
    def create_graph(self):
        """Crée un graphe à partir des données formatées."""
        for _, row in self.df.iterrows():
            self.graph.add_edge(
                row["departure_city"], 
                row["arrival_city"], 
                weight=row["duree"]
            )
    
    def find_shortest_path(self, start, end):
        """Trouve le chemin le plus court entre deux villes."""
        try:
            path = nx.shortest_path(self.graph, source=start, target=end, weight="weight")
            print(f"Chemin le plus court de {start} à {end} : {path}")
            return path
        except nx.NetworkXNoPath:
            print(f"Aucun chemin trouvé de {start} à {end}.")
            return None
    
    def run(self, predefined_start=None, predefined_end=None):
        """Exécute la recherche des chemins, avec des options de test prédéfinies."""
        self.create_graph()
        if predefined_start and predefined_end:
            print(f"Test automatique avec : départ = {predefined_start}, arrivée = {predefined_end}")
            self.find_shortest_path(predefined_start, predefined_end)
        else:
            while True:
                start = input("Entrez la ville de départ (ou 'exit' pour quitter) : ")
                if start.lower() == 'exit':
                    break
                end = input("Entrez la ville d'arrivée : ")
                self.find_shortest_path(start, end)

if __name__ == "__main__":
    formatter = FormatTimetables()
    formatter.format_timetables()
    
    pathfinder = PathFinder()
    pathfinder.run(predefined_start=PREDEFINED_START, predefined_end=PREDEFINED_END)

Aucune correspondance trouvée pour la gare : Gare de Le Havre
Aucune correspondance trouvée pour la gare : Gare de Paris-St-Lazare
Aucune correspondance trouvée pour la gare : Gare de Dieppe
Aucune correspondance trouvée pour la gare : Gare de Paris-St-Lazare
Aucune correspondance trouvée pour la gare : Gare de Paris-St-Lazare
Aucune correspondance trouvée pour la gare : Gare de Rouen-Rive-Droite
Aucune correspondance trouvée pour la gare : Gare de Cherbourg
Aucune correspondance trouvée pour la gare : Gare de Paris-St-Lazare
Aucune correspondance trouvée pour la gare : Gare de Caen
Aucune correspondance trouvée pour la gare : Gare de Paris-St-Lazare
Aucune correspondance trouvée pour la gare : Gare de Paris-St-Lazare
Aucune correspondance trouvée pour la gare : Gare de Trouville-Deauville
Aucune correspondance trouvée pour la gare : Gare de Granville
Aucune correspondance trouvée pour la gare : Gare de Paris-Montp.3-Vaug.
Aucune correspondance trouvée pour la gare : Gare de Granville


NodeNotFound: Source Le Havre is not in G